In [2]:
from buildstock_query import BuildStockQuery, ExeId, MappedColumn
import pandas as pd
import sqlalchemy as sa

In [3]:
print("hellodf")

hellodf


In [4]:
my_run = BuildStockQuery(db_name='buildstock_testing',
                        table_name='res_n250_hrly_v19',
                        workgroup='eulp',
                        buildstock_type='resstock',
                        skip_reports=False)

INFO:buildstock_query.query_core:Loading res_n250_hrly_v19 ...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.main:Getting Success counts...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...


         fail  unapplicable  success  Sum  Applied %  no-chng  bad-chng  \
upgrade                                                                   
0          14             0      236  250        0.0        0         0   
1           0           125      111  236       47.0        0         0   
2          10           185       41  236       21.6        9         1   
3           0             0      236  236      100.0        0         0   
4           0           236        0  236        0.0        0         0   
5         236             0        0  236      100.0        0         0   
6           0             0      236  236      100.0        0         0   
7          22           214        0  236        9.3        0         0   
8           0           194       42  236       17.8        0         1   

         ok-chng  true-bad-chng  true-ok-chng  null  any  no-chng %  \
upgrade                                                               
0              0              0 

INFO:buildstock_query.query_core:12 queries cache saved to .bsq_cache/res_n250_hrly_v19_query_cache.pkl


All buildings are verified to have the same number of (8760) timeseries rows.


In [5]:
query = my_run.get_building_ids(get_query_only=True)
query

'SELECT res_n250_hrly_v19_baseline.building_id \nFROM res_n250_hrly_v19_baseline'

In [6]:
building_type_map = {"Mobile Home": "MH", "Single-Family Detached": "SF",
                     "Single-Family Attached": "SF", "Multi-Family with 2 - 4 Units": "MF",
                     "Multi-Family with 5+ Units": "MF",}
bldg_col = my_run._get_column('build_existing_model.geometry_building_type_recs')
simple_bldg_col = MappedColumn(bsq=my_run, name='simple_bldg_type', mapping_dict=building_type_map,
                               key=bldg_col)
mapped_agg_query = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=[simple_bldg_col],
                                          get_query_only=False)
print(mapped_agg_query)

  simple_bldg_type  sample_count  units_count  \
0               MF            30     0.000056   
1               SF           206     0.000387   

   fuel_use_electricity_total_m_btu  
0                          0.001084  
1                          0.012372  


In [7]:
dryer_co2_impact = {"Gas, 120% Usage": 1.2 * 1.5, 
                    "Gas, 100% Usage": 1 * 1.5,
                    "Gas, 80% Usage": 0.8 * 1.5,
                    "Electric, 120% Usage": 1.2 * 0.5,
                    "Electric 100% Usage": 1 * 0.5,
                    "Electric 80% Usage": 0.8 * 0.5,
                    "None": 0}
bldg_col = my_run._get_column('build_existing_model.clothes_dryer')
impact_col = MappedColumn(bsq=my_run, name='dryer_impact', mapping_dict=dryer_co2_impact,
                               key=bldg_col)
mapped_agg_query = my_run.agg.aggregate_annual(enduses=[impact_col],
                                               get_query_only=False)
print(mapped_agg_query)

   sample_count  units_count  dryer_impact
0           236     0.000443       0.00021


In [8]:
baseline_agg_query = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                          get_query_only=True,
                                          )
print(baseline_agg_query)



SELECT res_n250_hrly_v19_baseline."build_existing_model.geometry_building_type_recs" AS geometry_building_type_recs, sum(1) AS sample_count, sum(res_n250_hrly_v19_baseline."build_existing_model.sample_weight") AS units_count, sum(res_n250_hrly_v19_baseline."report_simulation_output.fuel_use_electricity_total_m_btu" * res_n250_hrly_v19_baseline."build_existing_model.sample_weight") AS fuel_use_electricity_total_m_btu 
FROM res_n250_hrly_v19_baseline 
WHERE res_n250_hrly_v19_baseline.completed_status = 'Success' GROUP BY 1 ORDER BY 1


In [10]:
baseline_agg = my_run.execute(baseline_agg_query)
baseline_agg

,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu
0,Multi-Family with 2 - 4 Units,6,0.000011,0.000219
1,Multi-Family with 5+ Units,24,0.000045,0.000864
2,Single-Family Attached,3,0.000006,0.000114
3,Single-Family Detached,203,0.000381,0.012258


In [11]:
baseline_agg_query2 = my_run.agg.aggregate_annual(enduses=['fuel_use_natural_gas_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                          get_query_only=True)
print(baseline_agg_query2)

SELECT res_n250_hrly_v19_baseline."build_existing_model.geometry_building_type_recs" AS geometry_building_type_recs, sum(1) AS sample_count, sum(res_n250_hrly_v19_baseline."build_existing_model.sample_weight") AS units_count, sum(res_n250_hrly_v19_baseline."report_simulation_output.fuel_use_natural_gas_total_m_btu" * res_n250_hrly_v19_baseline."build_existing_model.sample_weight") AS fuel_use_natural_gas_total_m_btu 
FROM res_n250_hrly_v19_baseline 
WHERE res_n250_hrly_v19_baseline.completed_status = 'Success' GROUP BY 1 ORDER BY 1


In [12]:
bq = my_run.submit_batch_query([baseline_agg_query, baseline_agg_query2])
print(bq)
my_run.get_batch_query_report(bq)

1


INFO:buildstock_query.query_core:Submitted queries[0] (CACHED)


{'submitted': 0, 'running': 0, 'pending': 2, 'completed': 0, 'failed': 0}

INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.query_core:Submitted queries[1] (0ed33622-989f-44e5-b590-32493f804c37)


In [13]:
my_run.get_batch_query_report(bq)

INFO:botocore.tokens:Loading cached SSO token for nrel-sso


{'submitted': 2, 'running': 0, 'pending': 0, 'completed': 2, 'failed': 0}

In [14]:
result_df = my_run.get_batch_query_result(bq, combine=True)
result_df

INFO:buildstock_query.query_core:{'submitted': 2, 'running': 0, 'pending': 0, 'completed': 2, 'failed': 0}
INFO:buildstock_query.query_core:Batch query completed. 
INFO:buildstock_query.query_core:Got result from Query [0] (CACHED)
INFO:buildstock_query.query_core:Got result from Query [1] (0ed33622-989f-44e5-b590-32493f804c37)
INFO:buildstock_query.query_core:0ed33622-989f-44e5-b590-32493f804c37 cost 0.0 GB ($0.0). Session total: 0.0 GB ($0.0)
INFO:buildstock_query.query_core:Concatenating the results.


,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu,query_id,fuel_use_natural_gas_total_m_btu
0,Multi-Family with 2 - 4 Units,6,0.000011,0.000219,0,NaN
1,Multi-Family with 5+ Units,24,0.000045,0.000864,0,NaN
2,Single-Family Attached,3,0.000006,0.000114,0,NaN
3,Single-Family Detached,203,0.000381,0.012258,0,NaN
0,Multi-Family with 2 - 4 Units,6,0.000011,NaN,1,0.001069
1,Multi-Family with 5+ Units,24,0.000045,NaN,1,0.000719
2,Single-Family Attached,3,0.000006,NaN,1,0.000373
3,Single-Family Detached,203,0.000381,NaN,1,0.027768


In [15]:
query1 = my_run.agg.aggregate_annual(enduses=['fuel_use_natural_gas_total_m_btu'], group_by=['state'],
                                              get_query_only=True)
query2 = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                              group_by=['state'],
                                              get_query_only=True)
exe_id1, future1 = my_run.execute(query1, run_async=True)
exe_id2, future2 = my_run.execute(query2, run_async=True)

assert exe_id1 != "CACHED"  # Commenting this line will cause the type checker to complain on the next line
my_run.get_query_output_location(exe_id1)

's3://eulp/athena_query_results/cc974c2f-98fd-4657-bc21-45663f30beb3.csv'

In [16]:
future1.as_pandas()

,state,sample_count,units_count,fuel_use_natural_gas_total_m_btu
0,AL,2,0.000004,0.000074
1,AR,1,0.000002,0.000065
2,AZ,6,0.000011,0.000107
3,CA,23,0.000043,0.001449
4,CO,4,0.000008,0.000704
5,CT,3,0.000006,0.000296
6,DC,1,0.000002,0.000000
7,DE,1,0.000002,0.000267
8,FL,18,0.000034,0.000110
9,GA,5,0.000009,0.000410


In [17]:
my_run.get_query_status(ExeId(exe_id1))

'SUCCEEDED'

In [17]:
# my_run.get_athena_query_result(exe_id1)


In [18]:
my_run.get_all_running_queries()

[]

In [19]:
my_run.stop_all_queries()

INFO:buildstock_query.query_core:Stopped 0 queries


In [20]:
my_run.stop_query(ExeId(exe_id1))

{'ResponseMetadata': {'RequestId': '2b34fdc1-3a41-4904-b09b-6185bee959cb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 08 May 2024 22:03:02 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2b34fdc1-3a41-4904-b09b-6185bee959cb'},
  'RetryAttempts': 0}}

In [23]:
my_run.delete_everything()

Will delete the following tables ['res_test_03_2018_10k_20220607_baseline', 'res_test_03_2018_10k_20220607_timeseries', 'res_test_03_2018_10k_20220607_upgrades'] and the s3:/euss/tests/res_test_03_2018_10k_20220607 folder
Abandoned the idea.


In [20]:
# Mix and match: baseline from a different run from upgrades
my_run2 = BuildStockQuery(db_name='largeee_test_runs',
                          workgroup='largeee',
                          table_name=("baseline_20230525_baseline","cateory_2_20230525_timeseries","cateory_2_20230525_upgrades"),
                          buildstock_type='resstock',
                          skip_reports=False)


INFO:buildstock_query.query_core:Loading ('baseline_20230525_baseline', 'cateory_2_20230525_timeseries', 'cateory_2_20230525_upgrades') ...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.main:Getting Success counts...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...


         fail  unapplicable  success   Sum  Applied %  no-chng  bad-chng  \
upgrade                                                                    
0           7             0     9931  9938        0.0        0         0   
1           0             0     9931  9931      100.0        4       149   
2           0             0     9931  9931      100.0        3       391   
3           0          1793     8138  9931       81.9        4        16   
4           0           322     9609  9931       96.8      123        17   
5           0             0     9931  9931      100.0        2       222   

         ok-chng  true-bad-chng  true-ok-chng  null   any  no-chng %  \
upgrade                                                                
0              0              0             0     0     0        0.0   
1           9778            112          9815     0  9931        0.0   
2           9537            292          9637     0  9931        0.0   
3           8118              6

INFO:buildstock_query.query_core:12 queries cache saved to .bsq_cache/8d909ebc61709306982e1f11763d1184dfe3750c16c62b6aa305c9171b71760e_query_cache.pkl


All buildings are verified to have the same number of (8760) timeseries rows.
